Levantar servicio vllm:

- vllm serve --config config.yaml

Para levantar por ejemplo en una gpu y que solo aparezca en el sistema que tiene una GPU, esto para levantar un servicio o modelo por GPU.

- CUDA_VISIBLE_DEVICES=0 vllm serve --config config-single-gpu.yaml

y en la config ir cambiando el puerto.

In [1]:
!which python

/root/bert_ft_marcos/vllm_experiments/.venv/bin/python


In [ ]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8010/v1", api_key="EMPTY")

system_prompt = {
    "role": "system",
    "content": f"""Eres un amable asistente.
    Reasoning: Medium.
    """,
}

result = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[
        system_prompt,
        {
            "role": "user",
            "content": "Que vino primero el huevo o la gallina, explica brevemente.",
        },
    ],
)

In [4]:
print(result)

In [7]:
print(result.choices[0].message.content)

Los metadatos de las interacciones de las llamadas al ECU 911 hasta ahora son:
- Actor: [Operador, Alertante]
    - Etiquetados manualmente
- Acto de diálogo: [Pregunta, Descripción, Orden, Ruido, Rutina]
    - Etiquetados manualmente
Ahora la idea es usar este LLM para hacer estas cosas:
- POS
    - Obtenido con spacy (modelo transformer)
- Ner
    - Obtenido con spacy (modelo lg, el de mayor precisión)

Lo que falta:
- Subtipo o Subacto de diálogo, por ejemplo, si es pregunta: ["localización", "contacto", etc]
    - Aquí si propongo directo usar el LLM.
- Resumen del turno o interacción
    - Aquí se podría usar algún modelo ligero o sino para más precisión el mismo LLM.
- Términos clave
    - Aquí se puede utilizar el modelo que tengo en huggingface o sino de una vez utilizar el mismo LLM para mayor precisión.

In [2]:
# Lo que voy hacer es ir preguntando al LLM, cada interacción se va
# a enviar así:

# Prompt del sistema
system_prompt = {
    "role": "system",
    "content": """
You are an expert assistant specialized in analyzing spanish emergency call transcripts from the ECU911 system.
Your task is to analyze each dialogue turn and return structured JSON output.

### TASKS
Given a full call transcript, a dialogue turn (dialogue_turn) and its `dialog_act` label, you must produce:
1. "SUBTIPO_ACT": the subtype of the dialogue act, using only the allowed categories.
2. "RESUMEN_TURNO": a very short (1-line) summary of the turn in spanish.
3. "PALABRAS_CLAVE": a list of the most relevant terms in spanish from the turn (0–5 items). If no relevant terms, answer "-"

### ALLOWED SUBACTS (SUBTIPO_ACT)

If dialog_act = "Pregunta":
- "localización" e.g. ¿Dónde ocurrió?, ¿En qué calle?
- "heridos" e.g. ¿Hay personas heridas?, ¿Cuántos afectados?, ¿Cuántos heridos son?
- "tipo_incidente" e.g. ¿Es incendio, accidente, médico…? ¿Que pasó?
- "contacto" e.g. ¿Cuál es su nombre?, ¿Desde qué número llama?
- "confirmación" e.g. ¿Me confirma que hay humo negro?
- "other"

If dialog_act = "Descripción":
- "localización" e.g. Estamos en la Av. Amazonas y Patria
- "heridos" e.g. Hay dos personas inconscientes, Hay tres sospechosos
- "tipo_incidente" e.g. Es un incendio en una vivienda
- "síntomas" e.g. Le duele el pecho y no puede respirar
- "evolución" e.g. El fuego está creciendo rápido
- "other"

If dialog_act = "Orden":
- "permanecer_línea" e.g. Quédese en la línea, no cuelgue
- "acción_seguridad" e.g. Aléjese del lugar, no toque nada
- "seguir_instrucciones" e.g. Respire profundo y cuente hasta diez
- "otro"

If dialog_act = "Ruido":
- "ruido" e.g. Eh… sí… [ruido de sirena]… sí sí sí…

If dialog_act = "Rutina":
- "rutina_operador" e.g. cual es su emergencia Ecu 911 cuál es su emergencia, coordinamos su emergencia..
- "cortesía" e.g. muchas gracias, ya, muy amable, hasta luego
- "otro"

### OUTPUT RULES
- Always return **valid JSON only**. No explanations.
- Do not hallucinate information not present in the text.
- The summary must be concise, neutral, and professional.
- Keywords must be meaningful and relevant to emergency-response context.

### OUTPUT FORMAT
{
  "subact": "<SUBTIPO_ACT>",
  "turn_summary": "<summary>",
  "keywords": ["term1", "term2"]
}

Reasoning: Medium.
"""
}

# Prompt del usuario
# aquí sigo pensando si en cada interacción enviar la transcripción completa o no
# por ahora el prompt del sistema está hecho para aceptar la conversación completa



In [3]:
# cargar dataset
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_excel("../ner_pos_tf_idf/250_conversaciones_ner_pos.xlsx")

In [ ]:
# df = df[:10]

In [4]:
# ahora voy a cargar el dataset pero lo voy a unificar por TRA_ID
# para obtener la conversación completa.
df2 = df.copy()
df2["ATA_TEXTO_TAGGED"] = df2["ACTOR"] + ": " + df2["ATA_TEXTO_TAGGED"]
df2 = df2.groupby(["TRA_ID"], as_index=False).agg(
    {
        "ATA_TEXTO_TAGGED": " ".join 
    }
)
#a = df2[df2["TRA_ID"]==4365]["ATA_TEXTO_TAGGED"].iloc[0]
df2["LEN"] = df2["ATA_TEXTO_TAGGED"].str.split().str.len()
print(df2["LEN"].max())
print(df2["LEN"].min())


293
89


In [5]:
def search_call(tra_id):
    return df2[df2["TRA_ID"]==tra_id]["ATA_TEXTO_TAGGED"].iloc[0]

In [6]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8010/v1", api_key="EMPTY")

def get_llm_answer(user_prompt):
    
    result = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=[
            system_prompt,
            {
                "role": "user",
                "content": user_prompt
            },
        ],
    )
    
    return (result.choices[0].message.content)

In [9]:
def send_turn(row):
    content = f"""
        Full call spanish transcript:
        {search_call(row['TRA_ID'])}
        
        Analyze the following dialogue turn:
        dialogue_turn: "{row['ATA_TEXTO_TAGGED']}"
        actor: "{row['ACTOR']}"
        dialog_act: "{row['DIALOG_ACT']}"
        
        Return ONLY the JSON.
        """
    
    
    return get_llm_answer(content)
    

In [10]:
df["llm_output"] = df.progress_apply(send_turn, axis=1)

100%|█████████████████████████████████████████████████████████████████| 5052/5052 [4:52:17<00:00,  3.47s/it]


In [9]:
df

,TRA_ID,INCIDENTGRADENAME,ORDER,ATA_TEXTO_TAGGED,ACTOR,DIALOG_ACT,NER,POS,llm_output
0,4365,CLAVE ROJA,1,emergencia,Operador,Pregunta,[],"[('emergencia', 'NOUN')]","{\n ""subact"": ""other"",\n ""turn_summary"": ""El..."
1,4365,CLAVE ROJA,2,buenos dias hagame un favor me puede mandar un...,Alertante,Pregunta,"[('buenos dias hagame', 'MISC')]","[('buenos', 'ADJ'), ('dias', 'NOUN'), ('hagame...","{\n ""subact"": ""other"",\n ""turn_summary"": ""So..."
2,4365,CLAVE ROJA,3,que sucede señore que le sucede,Operador,Pregunta,"[('señore', 'PER')]","[('que', 'SCONJ'), ('sucede', 'VERB'), ('señor...","{""subact"":""other"",""turn_summary"":""El operador ..."
3,4365,CLAVE ROJA,4,esta un carro parqueado y un par de jovenes co...,Alertante,Descripción,[],"[('esta', 'VERB'), ('un', 'DET'), ('carro', 'N...","{\n ""subact"": ""tipo_incidente"",\n ""turn_summ..."
4,4365,CLAVE ROJA,5,le agredieron,Operador,Pregunta,[],"[('le', 'PRON'), ('agredieron', 'VERB')]","{\n ""subact"": ""confirmación"",\n ""turn_summar..."
5,4365,CLAVE ROJA,6,a mi no me rompen el parabrisas del carro que ...,Alertante,Descripción,[],"[('a', 'ADP'), ('mi', 'PRON'), ('no', 'ADV'), ...","{\n ""subact"": ""tipo_incidente"",\n ""turn_summ..."
6,4365,CLAVE ROJA,7,cuantos personas son,Operador,Pregunta,[],"[('cuantos', 'DET'), ('personas', 'NOUN'), ('s...","{\n ""subact"": ""heridos"",\n ""turn_summary"": ""..."
7,4365,CLAVE ROJA,8,dos jovenes dos jovenes,Alertante,Descripción,[],"[('dos', 'NUM'), ('jovenes', 'NOUN'), ('dos', ...","{\n ""subact"": ""other"",\n ""turn_summary"": ""El..."
8,4365,CLAVE ROJA,9,como estan vestidos señora,Operador,Pregunta,[],"[('como', 'SCONJ'), ('estan', 'AUX'), ('vestid...","{\n ""subact"": ""other"",\n ""turn_summary"": ""Pr..."
9,4365,CLAVE ROJA,10,no se apretaron a correr oiga,Alertante,Descripción,[],"[('no', 'ADV'), ('se', 'PRON'), ('apretaron', ...","{\n ""subact"": ""evolución"",\n ""turn_summary"":..."


In [11]:
df.to_excel("raw_llm_results.xlsx", index=False)

In [13]:
# procesar las respuestas del LLM con json repair

import pandas as pd
import json_repair

def parse_llm_output(text):

    if not isinstance(text, str) or text.strip() == "":
        return pd.Series([None, None, []])

    try:
        obj = json_repair.repair_json(
            text,
            return_objects=True,
            ensure_ascii=False
        )

        # Si no devuelve un dict válido
        if not isinstance(obj, dict):
            return pd.Series([None, None, []])

        subact = obj.get("subact")
        turn_summary = obj.get("turn_summary")
        keywords = obj.get("keywords", [])

        # Asegurar tipos esperados
        if not isinstance(keywords, list):
            keywords = []

        return pd.Series([subact, turn_summary, keywords])

    except Exception:
        return pd.Series([None, None, []])


df[["SUBACT", "TURN_SUMMARY", "KEYWORDS"]] = df["llm_output"].apply(parse_llm_output)


In [5]:
# reemplazar ner en el texto

import pandas as pd

df = pd.read_excel("processed_llm_output.xlsx")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5052 entries, 0 to 5051
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   TRA_ID             5052 non-null   int64 
 1   INCIDENTGRADENAME  5052 non-null   object
 2   ORDER              5052 non-null   int64 
 3   ATA_TEXTO_TAGGED   5052 non-null   object
 4   ACTOR              5052 non-null   object
 5   DIALOG_ACT         5052 non-null   object
 6   NER                5052 non-null   object
 7   POS                5052 non-null   object
 8   llm_output         5052 non-null   object
 9   SUBACT             5052 non-null   object
 10  TURN_SUMMARY       5052 non-null   object
 11  KEYWORDS           5052 non-null   object
dtypes: int64(2), object(10)
memory usage: 473.8+ KB


In [7]:
import ast

def replace_ner(row):
    
    ners = ast.literal_eval(row["NER"])
    raw_text = str(row["ATA_TEXTO_TAGGED"])
    if len(ners) > 0:
        for ner in ners:
            raw_text = raw_text.replace(ner[0], f"[{ner[1]}]")
            
    return raw_text

df['ANONIMIZED_TEXT'] = df.apply(replace_ner, axis=1)

In [ ]:
#df.to_excel("processed_llm_output_anonimized.xlsx", index=False)

In [1]:
# limpiar un poco los subactos

import pandas as pd

df = pd.read_excel("processed_llm_output_anonimized.xlsx")

In [2]:
# Los subactos permitidos son:
# If dialog_act = "Pregunta":
# - "localización" e.g. ¿Dónde ocurrió?, ¿En qué calle?
# - "heridos" e.g. ¿Hay personas heridas?, ¿Cuántos afectados?, ¿Cuántos heridos son?
# - "tipo_incidente" e.g. ¿Es incendio, accidente, médico…? ¿Que pasó?
# - "contacto" e.g. ¿Cuál es su nombre?, ¿Desde qué número llama?
# - "confirmación" e.g. ¿Me confirma que hay humo negro?
# - "other"

# If dialog_act = "Descripción":
# - "localización" e.g. Estamos en la Av. Amazonas y Patria
# - "heridos" e.g. Hay dos personas inconscientes, Hay tres sospechosos
# - "tipo_incidente" e.g. Es un incendio en una vivienda
# - "síntomas" e.g. Le duele el pecho y no puede respirar
# - "evolución" e.g. El fuego está creciendo rápido
# - "other"

# If dialog_act = "Orden":
# - "permanecer_línea" e.g. Quédese en la línea, no cuelgue
# - "acción_seguridad" e.g. Aléjese del lugar, no toque nada
# - "seguir_instrucciones" e.g. Respire profundo y cuente hasta diez
# - "otro"

# If dialog_act = "Ruido":
# - "ruido" e.g. Eh… sí… [ruido de sirena]… sí sí sí…

# If dialog_act = "Rutina":
# - "rutina_operador" e.g. cual es su emergencia Ecu 911 cuál es su emergencia, coordinamos su emergenica..
# - "cortesía" e.g. muchas gracias, ya, muy amable, hasta luego
# - "otro"

In [3]:
# los subactos del dataframe son:
df["SUBACT"].unique()

array(['other', 'tipo_incidente', 'heridos', 'seguir_instrucciones',
       'localización', 'otro', 'localizacion', 'contacto',
       'rutina_operador', 'cortesía', 'síntomas', 'confirmación',
       'evolución', 'ruido', 'permanecer_línea', 'acción_seguridad',
       'confirmacion', 'contact', 'localization'], dtype=object)

In [ ]:
# hay que unificar "other" con "otro", solo quedarse con "otro"
# hay que unificar "localizacion" con "localización" y "localization", quedarse con "localización"
# hay que unificar "contacto" con "contact", quedarse con "contacto"
# hay que unificar "confirmación" con "confirmacion", quedarse con "confirmación"

# df["SUBACT"] = df["SUBACT"].str.replace("other", "otro")
# df["SUBACT"] = df["SUBACT"].str.replace("localizacion", "localización")
# df["SUBACT"] = df["SUBACT"].str.replace("localization", "localización")
# df["SUBACT"] = df["SUBACT"].str.replace("contact", "contacto")
# df["SUBACT"] = df["SUBACT"].str.replace("confirmacion", "confirmación")

In [4]:
subact_mapping = {
    "other": "otro",
    "localizacion": "localización",
    "localization": "localización",
    "contact": "contacto",
    "confirmacion": "confirmación",
}

df["SUBACT"] = df["SUBACT"].replace(subact_mapping)

In [5]:
df.to_excel("processed_llm_output_anonimized_clean.xlsx", index=False)